Image service currently works with pre-computed MediaLibraries that contain images. 

The images are generated from _tags_, such as character name, a scene, time of the day, etc. These tags can be later used to filter the desired images. 

In [1]:
models = {}
character = {}
negative = {}
general = {}

general['Lina'] = '(masterpiece:1.2), best quality, high resolution, unity 8k wallpaper, (illustration:0.8), (beautiful detailed eyes:1.6), extremely detailed face, perfect lighting, extremely detailed CG, (perfect hands, perfect anatomy)'
models['Lina'] = 'models\\Stable-diffusion\\anyloraCheckpoint_novaeFp16.safetensors'
character['Lina'] = 'dotalina, <lora:lina-lora-nochekaiser:0.7>, lina'


general['Jug'] = 'detailed texture, ultra detailed, best quality, highres, photorealistic, raw, 8k,masterpiece, masterpiec8K.HDR'
models['Jug'] = 'models\\Stable-diffusion\\dreamshaper_8.safetensors'
character['Jug'] = '<lora:jugg:0.8> jugg, mask, upper body image'

negative['Jug'] = negative['Lina'] = '(worst quality, low quality:1.4), (jpeg artifacts:1.4),greyscale, monochrome, motion blur, emphasis lines, text, title, logo, signature,bad_hands, bad eyes, bad-artist-anime, easynegative:1.3, slanted eyes'

views = ['front view', 'side view']

scenes = {
    'outside': 'epic scene, outside, sundown, beautiful nature',
    'magic': 'doing magic, fires, sparks, fire show',
}

Out tags will be character (`lina`) and the keys of `scenes` dictionary. First, we will use `Template` to transform tags to a prompt.

In [2]:
from kaia.avatar.dub.core import Template, ToStrDub, DictDub, IdentityDub, DictBinding

image_prompt=Template(
    "{general}\n{character}\n{scene}\n{view}",
    general = DictBinding('character', general),
    character = DictDub(character),
    scene = DictDub(scenes),
    view = IdentityDub()
)

print(image_prompt.to_str(dict(character='Lina', scene='magic', view='front view')))

(masterpiece:1.2), best quality, high resolution, unity 8k wallpaper, (illustration:0.8), (beautiful detailed eyes:1.6), extremely detailed face, perfect lighting, extremely detailed CG, (perfect hands, perfect anatomy)
dotalina, <lora:lina-lora-nochekaiser:0.7>, lina
doing magic, fires, sparks, fire show
front view


Now, we use `PictureTaskGenerator` to transform prompt to task:

In [3]:
from kaia.avatar.narrator import PictureTaskGenerator
from kaia.brainbox import BrainBoxTask
from pprint import pprint

generator = PictureTaskGenerator(
    prompt_template = image_prompt,
    brain_box_task_template = BrainBoxTask.template(
        decider='Automatic1111', 
        arguments=dict(batch_size=2), 
    ),
    collecting_task_template = BrainBoxTask.template(decider='Collector'),
    values_to_model = lambda v: models[v['character']],
    values_to_negative = lambda v: models[v['character']]
)

pprint(generator.generate_task(dict(character='Lina', scene='magic', view='front view')).__dict__)

{'arguments': {'batch_size': 2,
               'negative_prompt': 'models\\Stable-diffusion\\anyloraCheckpoint_novaeFp16.safetensors',
               'prompt': '(masterpiece:1.2), best quality, high resolution, '
                         'unity 8k wallpaper, (illustration:0.8), (beautiful '
                         'detailed eyes:1.6), extremely detailed face, perfect '
                         'lighting, extremely detailed CG, (perfect hands, '
                         'perfect anatomy)\n'
                         'dotalina, <lora:lina-lora-nochekaiser:0.7>, lina\n'
                         'doing magic, fires, sparks, fire show\n'
                         'front view'},
 'back_track': None,
 'batch': None,
 'decider': 'Automatic1111',
 'decider_method': None,
 'decider_parameters': 'models\\Stable-diffusion\\anyloraCheckpoint_novaeFp16.safetensors',
 'dependencies': None,
 'id': 'id_f913dbc7e1544008b109d71a679285b7'}


Now we build a task pack:

In [4]:
tag_values = [
    dict(character=c, scene=s, view=v)
    for c in general
    for s in scenes
    for v in views
]

pack = generator.generate_task_pack(tag_values)

In [5]:
import os
from pathlib import Path
from kaia.brainbox import BrainBox, MediaLibrary

media_library_path = Path('files/image_library.zip')

os.makedirs(media_library_path.parent, exist_ok=True)
if not media_library_path.is_file():
    with BrainBox().create_test_api() as api:
        lib = api.execute(pack)
        if len(lib.records)>0:
            lib.save(media_library_path)
else:
    lib = MediaLibrary.read(media_library_path)

In [6]:
from ipywidgets import Image, HBox
from yo_fluq_ds import *

d = lib.to_dict()
df = lib.to_df().feed(fluq.add_ordering_column(['character','scene'], 'filename')).feed(lambda z: z.loc[z.order==0])
Query.en(df.filename).select(lambda z: Image(value = d[z].get_content())).feed(list, HBox)


These images aren't of the best quality (probably because the used Lora isn't, and because I didn't really pay much of attention to the prompt). However, they are suitable for the following demonstration.